
# Chapter 6: LangChain and Generative Workflows

This notebook covers:
- LangChain architecture: Chains, Agents, Memory
- How to use document loaders and text splitters
- Connecting retrievers and vector stores
- Designing multi-step workflows with LangChain
- Deploying via LangServe

## Learning Objectives

- Understand the components of LangChain: Prompt, LLM, Chain, Agent
- Load and split documents for context-aware processing
- Connect vector databases like FAISS with LangChain retrievers
- Build multi-step chains and agent tools



## LangChain Components Overview

| Component | Purpose |
|----------|---------|
| PromptTemplate | Parameterized prompt templates |
| LLMChain | Chains that use LLMs |
| Memory | Store interaction history |
| Retriever | Fetch context from knowledge base |
| Agent | Execute reasoning with tools |


In [ ]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI  # Replace with local model if needed

# Create a prompt template
template = PromptTemplate.from_template("Translate the following to Spanish: {text}")
llm = OpenAI(temperature=0.7)  # Requires API key
chain = LLMChain(prompt=template, llm=llm)

# Run the chain
output = chain.run("Good morning, how are you?")
print(output)



## Document Loading and Splitting

LangChain supports loaders (PDF, web, TXT) and splitters for chunking text.


In [ ]:

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader(file_path="sample.txt")  # Replace with a valid file path
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
print(f"Loaded {len(split_docs)} chunks.")



## Retriever + VectorStore Integration

You can embed documents and retrieve them for query answering.


In [ ]:

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

texts = ["LangChain enables generative workflows.", "Retrievers fetch relevant information for answering."]
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

db = FAISS.from_texts(texts, embeddings)
retriever = db.as_retriever()

query = "What is LangChain?"
docs = retriever.get_relevant_documents(query)
print("Top document retrieved:", docs[0].page_content)



## Multi-Step Generative Workflow

Example: Retrieve → Summarize → Refine Answer


In [ ]:

from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate

# First: Summarize
summarize_prompt = PromptTemplate.from_template("Summarize: {text}")
summarize_chain = LLMChain(llm=llm, prompt=summarize_prompt)

# Second: Rephrase
refine_prompt = PromptTemplate.from_template("Rephrase more formally: {text}")
refine_chain = LLMChain(llm=llm, prompt=refine_prompt)

# Combine
workflow = SimpleSequentialChain(chains=[summarize_chain, refine_chain])
result = workflow.run("LangChain is a cool framework to connect LLMs.")
print("Final output:", result)



## Deploying with LangServe

LangChain supports deployment of chains using LangServe + FastAPI.

```bash
pip install langserve
```

```python
from langserve import add_routes
from fastapi import FastAPI

app = FastAPI()
add_routes(app, chain=your_chain, path="/generate")
```

Then launch with `uvicorn main:app --reload`



## Exercises

1. Load a Wikipedia article and chunk it using LangChain.
2. Index your documents in FAISS and build a custom retriever.
3. Create a multi-step workflow using 3 chains.
4. Use LangServe to expose a chatbot chain as an API.

## References

- LangChain Docs: https://docs.langchain.com
- LangServe: https://github.com/langchain-ai/langserve
- FAISS: https://github.com/facebookresearch/faiss
